In [1]:
#some basic imports and setups
import os
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
from difflib import SequenceMatcher



#mean of imagenet dataset in BGR
imagenet_mean = np.array([104., 117., 124.], dtype=np.float32)

current_dir = os.getcwd()
image_dir = os.path.join(current_dir, 'memes')
#image_dir = current_dir

%matplotlib inline

Converting captions and meme vector representations into single Tfrecord

Requires putting memes through alexnet to find their vector rep, shuffling the captions, changing captions into their word2idx, finally saving one caption together with one meme.

In [2]:
#USE FOR INCEPTION NETWORK (OR ALEXNET WITH FINETUNING) #NOW WITH LABELS INCLUDED
#TRAINING SET
#For this case just need to save image filenames alongside captions
with open('/Users/summ7t/MemeProject/im2txt/ordered_memes.txt','r') as f:
    img_files = f.readlines()
img_files = [os.path.join(image_dir, f) for f in img_files] # add path to each file
img_files = [img_file.replace('\n','') for img_file in img_files]
print(img_files[0],img_files[-1])
with open('/Users/summ7t/MemeProject/im2txt/CaptionsClean.txt','r') as f:
    captions = f.readlines()

data_memes = []
data_captions = []
data_labels = []
counter = 0
passed = 0


for i,meme in enumerate(img_files):
    
    match = []
    meme_name = captions[counter].split(' - ')[0]
    image_name = meme.replace('/Users/summ7t/MemeProject/im2txt/memes/','')
    image_name = image_name.replace('.jpg','')
    image_name = image_name.replace('-',' ')
    #print(meme_name,image_name)
    try:
        assert SequenceMatcher(a=meme_name.replace(' ',''),b=image_name.replace(' ','')).ratio() >= 0.75 or image_name in meme_name or meme_name in image_name
    except AssertionError:
        passed+=1
        continue
        
    while SequenceMatcher(a=meme_name.replace(' ',''),b=captions[counter].split(' - ')[0].replace(' ','')).ratio() >= 0.75 or meme_name in captions[counter].split(' - ')[0] or captions[counter].split(' - ')[0] in meme_name: 
        if counter==len(captions)-1:
            match.append(captions[counter].split(' - ')[-1])
            break
        elif captions[counter] == captions[counter].split(' - ')[-1]:
            counter += 1
        else:
            match.append(captions[counter].split(' - ')[-1])
            counter += 1
                
        
    #now save in tfrecords format, or prepare for that action
    meme_images = [meme for cap in match]
    meme_labels = [meme_name for cap in match]
    assert len(meme_images) == len(match)
    data_memes.extend(meme_images)
    data_captions.extend(match)
    data_labels.extend(meme_labels)

    if i % 100 == 0:
        print(i,len(data_memes),len(data_captions),len(meme_labels))
print(passed)
print(len(data_memes))
        

('/Users/summ7t/MemeProject/im2txt/memes/y-u-no.jpg', '/Users/summ7t/MemeProject/im2txt/memes/mendozameme.jpg')
(0, 194, 194, 194)
(100, 19500, 19500, 195)
(200, 38642, 38642, 186)
(300, 57715, 57715, 193)
(400, 76329, 76329, 193)
(500, 94590, 94590, 195)
(600, 113145, 113145, 193)
(700, 131506, 131506, 193)
(800, 149519, 149519, 188)
(900, 167517, 167517, 145)
(1000, 185424, 185424, 182)
(1100, 203293, 203293, 178)
(1200, 220335, 220335, 191)
(1300, 237822, 237822, 193)
(1400, 254128, 254128, 188)
(1500, 270057, 270057, 106)
(1600, 286621, 286621, 190)
(1700, 302199, 302199, 135)
(1800, 316651, 316651, 191)
(1900, 331885, 331885, 195)
(2000, 346318, 346318, 192)
(2100, 362088, 362088, 1)
(2200, 375125, 375125, 188)
(2300, 389122, 389122, 23)
(2400, 401659, 401659, 189)
(2500, 414388, 414388, 1)
23
414388


In [3]:
len(data_memes),len(data_captions),len(data_labels)

(414388, 414388, 414388)

In [4]:
for i in range(10,50):
    print(data_memes[i*100+402000])
    print(data_captions[i*100+402000])
    print(data_labels[i*100+402000])

/Users/summ7t/MemeProject/im2txt/memes/i-hardly-know-her.jpg
hebrew? i hardly knew you!

i hardly know her
/Users/summ7t/MemeProject/im2txt/memes/typical-grandmother.jpg
suor grazia

typical grandmother
/Users/summ7t/MemeProject/im2txt/memes/shabbywag.jpg
fuckshitbitch pissassdick

shabbywag
/Users/summ7t/MemeProject/im2txt/memes/facebook-roleplay-ocelot.jpg
i not understand your speking inglish rite?

facebook roleplay ocelot
/Users/summ7t/MemeProject/im2txt/memes/facebook-roleplay-ocelot.jpg
happy new year alex!! miss & love ya bunches!! <3

facebook roleplay ocelot
/Users/summ7t/MemeProject/im2txt/memes/funny-stupid.jpg
and make good choices

funny stupid
/Users/summ7t/MemeProject/im2txt/memes/unlucky-person.jpg
egd idag och imorgon

unlucky person
/Users/summ7t/MemeProject/im2txt/memes/indian-gangster-wannabe.jpg
my name is anoop aka anoop dogg

indian gangster wannabe
/Users/summ7t/MemeProject/im2txt/memes/indian-gangster-wannabe.jpg
gua miss sama lu ah anak ayamkuuu

indian gangs

In [5]:
data_labels[len(data_memes)-200]

'arashian alpaca'

In [6]:
#CREATE EVALUATION SET    #NOW WITH LABELS AS WELL
#SHOULD BE MEMES WITH REPEATED FORMAT 
counter = 0
R = []
idxs = []
for i,capt in enumerate(data_captions):
    if i % 190 == 0: 
        start = ' '.join(capt.split()[0:2])
        counter = 0
    if SequenceMatcher(a=start,b=' '.join(capt.split()[0:2])).ratio() >= 0.9:
        counter += 1 
    if counter >20 and start not in R:
        R.append(start)
        idxs.append(i)
print(len(R))

deleters = [0,1,3,16,79,99]
for i,ting in enumerate(deleters):
    del R[ting-i]
    del idxs[ting-i]
print(R)

103
['yo dawg', 'prepare yourselves', 'so your', 'not sure', 'guess who', 'you said', 'back in', "i'll have", 'yeah, if', 'you get', "i'll start", "so you'r", 'i dont', 'look marge', 'look at', 'in soviet', 'remeber when', 'if you', 'hey girl', "why can't", 'i thought', 'why the', 'i can', "why don't", 'my precious', 'this is', 'confucius say', 'brace yourselves', 'how tough', 'i see', 'what do', "i don't", 'and then', 'i said:', 'good good', 'si no', 'watch out', 'welcome to', 'yeah sure', 'i hate', 'yo mama', 'did somebody', 'everyone is', 'good morning', 'when you', 'its not', 'right in', 'you merely', 'remember when', 'hide yo', 'i fucking', 'happy birthday', 'niggas be', 'uy si', 'vote for', 'por los', 'nigga you', "and that's", 'no one', "if you're", 'oogah boogah', 'hoy en', 'you mean', 'oh so', "you don't", 'so you', 'er mah', 'hey guys', 'i heard', 'dafuq did', "you've just", 'there will', 'fuck you', 'i like', 'release the', 'you should', 'fuck it', 'no i', "i'm commander", '

In [7]:
eval_examples = [0,570,760,1140,2850,3040,4940,28500] + idxs
eval_captions = ['y u no','i dont always','not sure if','one does not simply','what if i told you','what if',"so you're telling me",'so then i said'] + R
eval_labels = []
eval_memes = []

for idx in eval_examples:
    eval_memes.append(data_memes[idx])
    eval_labels.append(data_labels[idx])


In [8]:
eval_labels

['y u no',
 'the most interesting man in the world',
 'futurama fry',
 'one does not simply',
 'what if i told you',
 'conspiracy keanu',
 'skeptical 3rd world kid',
 'so then i said...',
 'yo dawg',
 'prepare yourself',
 'skeptical african child',
 'not sure if troll',
 'guess who ?',
 'maury lie detector',
 '1889 [10] guy',
 'tough spongebob',
 'yeah if you could just',
 'oprah you get a',
 'bender blackjack and hookers',
 'skeptical third-world kid',
 "i don't know who you are...",
 'look-marge',
 'look at all the fucks i give',
 'in soviet russia',
 'pepperidge farm remembers fg',
 'office space boss',
 'ryan gosling hey girl 3',
 "why can't i hold all these?!?!?",
 'never have i been so wrong',
 'patrick stewart wtf',
 'i can haz',
 'pushing patrick',
 'my precious gollum',
 "timmy turner's dad if i had one!",
 'wise confucius',
 'brace yourselves.',
 'how tough are you',
 'i see dead people',
 'what do we say to the god of death ? ',
 'serj tankian',
 'reagan white house laughing

In [9]:
#Should also look at removing examples which are similar but not exactly the same ##WITH LABELS

c = list(zip(data_memes, data_captions, data_labels))
no_repeats = []
# order preserving
def idfun(x): return x

seen = {}
no_repeats = []
for item in c:
    marker = idfun(item[1])
    # in old Python versions:
    # if seen.has_key(marker)
    # but in new ones:
    if marker in seen: continue
    seen[marker] = 1
    no_repeats.append(item)
print(len(data_captions))
len(no_repeats)


414388


385707

In [10]:
from random import shuffle
shuffle(no_repeats)
memes_shuffled, captions_shuffled, labels_shuffled = zip(*no_repeats)
memes_shuffled = list(memes_shuffled)
captions_shuffled = list(captions_shuffled)
labels_shuffled = list(labels_shuffled)

In [11]:
import re
word_captions = []
for capt in captions_shuffled + labels_shuffled: #include labels_shuffled here for glove averages
    words = re.findall(r"[\w']+|[.,:!?;'><(){}%$#£@-_+=|\/~`^&*]", capt)
    word_captions.append(words)
print(len(word_captions))

771414


In [12]:
from collections import Counter
print("Creating vocabulary.")
counter = Counter()
for c in word_captions:
    counter.update(c)
print("Total words:", len(counter))

# Filter uncommon words and sort by descending count.
word_counts = [x for x in counter.items() if x[1] >= 3]
word_counts.sort(key=lambda x: x[1], reverse=True)
print("Words in vocabulary:", len(word_counts))

Creating vocabulary.
('Total words:', 150760)
('Words in vocabulary:', 41819)


In [13]:
# Create the vocabulary dictionary.
reverse_vocab = [x[0] for x in word_counts]
#unk_id = len(reverse_vocab)
vocab_dict = dict([(x, y) for (y, x) in enumerate(reverse_vocab)])

In [14]:
vocab_dict[':']

67

In [15]:
EMBEDDING_DIMENSION=300 # Available dimensions for 6B data is 50, 100, 200, 300
data_directory = current_dir

PAD_TOKEN = 0

word2idx = { 'PAD': PAD_TOKEN } # dict so we can lookup indices for tokenising our text later from string to sequence of integers
weights = []
index_counter = 0

with open('glove.42B.300d.txt','r') as file:
    for index, line in enumerate(file):
        values = line.split() # Word and weights separated by space
        word = values[0] # Word is first symbol on each line
        if word in vocab_dict:
            index_counter += 1
            word_weights = np.asarray(values[1:], dtype=np.float32) # Remainder of line is weights for word
            word2idx[word] = index_counter # PAD is our zeroth index so shift by one
            weights.append(word_weights)
        if index % 100000 == 0:
            print(index)
        if index + 1 == 2000000:
            break

EMBEDDING_DIMENSION = len(weights[0])
# Insert the PAD weights at index 0 now we know the embedding dimension
weights.insert(0, np.zeros(EMBEDDING_DIMENSION))

# Append unknown and pad to end of vocab and initialize as random #maybe include start and end token here
UNKNOWN_TOKEN=len(weights)
#include * token as its very common? why not
word2idx['*'] = UNKNOWN_TOKEN
word2idx['UNK'] = UNKNOWN_TOKEN + 1
word2idx['<S>'] = UNKNOWN_TOKEN + 2
word2idx['</S>'] = UNKNOWN_TOKEN + 3
weights.append(np.random.randn(EMBEDDING_DIMENSION)*0.5)
weights.append(np.random.randn(EMBEDDING_DIMENSION)*0.5)
weights.append(np.random.randn(EMBEDDING_DIMENSION)*0.5)
weights.append(np.random.randn(EMBEDDING_DIMENSION)*0.5)

# Construct our final vocab
weights = np.asarray(weights, dtype=np.float32)

VOCAB_SIZE=weights.shape[0]
print(VOCAB_SIZE)

#Save Vocabulary IF NEW
with tf.gfile.FastGFile('vocab_averages.txt', "w") as f:
    f.write("\n".join(["%s %d" % (w, c) for w, c in word2idx.iteritems()]))
print("Wrote vocabulary file:", 'vocab_averages.txt')

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
38821
Instructions for updating:
Use tf.gfile.GFile.
('Wrote vocabulary file:', 'vocab_averages.txt')


In [16]:
weights[37984]

array([ 0.045762 , -0.43857  ,  0.39753  , -0.028656 , -0.15955  ,
        0.045347 ,  0.63759  , -0.46284  ,  0.40802  ,  0.21301  ,
       -0.0675   , -0.14272  ,  0.086288 ,  0.010296 ,  0.051793 ,
        0.27871  ,  0.44854  , -0.12653  , -0.019418 ,  0.020812 ,
        0.089711 , -0.26724  , -0.26729  , -0.54687  , -0.23297  ,
       -0.01217  , -0.15458  , -0.31502  ,  0.43345  , -0.10731  ,
       -0.25131  ,  0.014674 , -0.011975 , -0.11912  ,  0.0054   ,
       -0.040286 ,  0.14262  , -0.0013592,  0.0027098, -0.093035 ,
        0.0027024, -0.35825  , -0.35138  , -0.038947 ,  0.276    ,
        0.23442  ,  0.12997  ,  0.59029  ,  0.16466  , -0.082049 ,
       -0.51823  , -0.28202  , -0.37064  , -0.0038544, -0.14939  ,
       -0.13021  , -0.049993 , -0.007823 , -0.0040152,  0.72491  ,
       -0.30818  , -0.21168  , -0.060183 ,  0.098535 ,  0.18045  ,
       -0.048824 ,  0.40144  , -0.087334 , -0.40455  ,  0.30746  ,
       -0.24522  , -0.085528 , -0.31777  , -0.044877 , -0.4234

In [17]:
word2idx['_']

766

In [18]:
#SAVE EMBEDDING MATRIX IF NEW
np.savetxt('embedding_matrix_averages',weights)

In [19]:
import re
token_captions = []
for capt in captions_shuffled:
    token_caption = []
    token_caption.append(word2idx['<S>'])
    words = re.findall(r"[\w']+|[.,:!?;'><(){}%$#£@-_+=|\/~`^&*]", capt)
    for word in words:
        try:
            token = word2idx[word]
        except KeyError:
            token = word2idx['UNK']
        token_caption.append(token)
    token_caption.append(word2idx['</S>'])
    token_captions.append(token_caption)
    
#for training labels
token_labels = []
for capt in labels_shuffled:
    token_caption = []
    words = re.findall(r"[\w']+|[.,:!?;'><(){}%$#£@-_+=|\/~`^&*]", capt)
    for word in words:
        try:
            token = word2idx[word]
        except KeyError:
            token = word2idx['UNK']
        token_caption.append(token)
    token_labels.append(token_caption)
    
#for eval set
eval_tokens = []
for capt in eval_captions:
    token_caption = []
    token_caption.append(word2idx['<S>'])
    words = re.findall(r"[\w']+|[.,:!?;'><(){}%$#£@-_+=|\/~`^&*]", capt)
    for word in words:
        try:
            token = word2idx[word]
        except KeyError:
            token = word2idx['UNK']
        token_caption.append(token)
    #token_caption.append(word2idx['</S>'])
    eval_tokens.append(token_caption)
    
#for eval labels
token_labels_eval = []
for capt in eval_labels:
    token_caption = []
    words = re.findall(r"[\w']+|[.,:!?;'><(){}%$#£@-_+=|\/~`^&*]", capt)
    for word in words:
        try:
            token = word2idx[word]
        except KeyError:
            token = word2idx['UNK']
        token_caption.append(token)
    token_labels_eval.append(token_caption)

In [20]:
print(token_captions[330000])
print(captions_shuffled[330000])
print(token_labels[330000])
print(labels_shuffled[330000])
print(eval_tokens[3])
print(eval_captions[3])
print(token_labels_eval[3])
print(eval_labels[3])
word2idx['UNK']

[38819, 4459, 34, 38818, 5036, 34, 38818, 50, 137, 220, 97, 16, 34, 33, 28, 7, 8226, 243, 4, 487, 34, 38820]
omg! we're careers! we're so much better than you! we have a lame name and everything!

[5036]
careers
[38819, 47, 138, 32, 717]
one does not simply
[47, 138, 32, 717]
one does not simply


38818

In [21]:
memes_shuffled = list(memes_shuffled)
captions_shuffled = list(captions_shuffled)

In [22]:
deleters = []
for i,ting in enumerate(token_captions):
    if len(ting) == 2:
        deleters.append(i)

In [23]:
for i,ting in enumerate(deleters):
    print(ting)
    del captions_shuffled[ting-i]
    del memes_shuffled[ting-i]
    del token_captions[ting-i]
    del labels_shuffled[ting-i]
    del token_labels[ting-i]
deleters = []
for i,ting in enumerate(token_captions):
    if len(ting) == 2:
        deleters.append(i)
len(deleters)

7833
9737
73843
78273
138696
224617
263114
279497
292947
297613
367050


0

In [24]:
many_unk = []
for i,capt in enumerate(token_captions):
    unk_counter = 0
    for token in capt:
        if token == word2idx['UNK']:
            unk_counter += 1
    if unk_counter >= 2:
        many_unk.append(i)
print(len(many_unk))

for i,ting in enumerate(many_unk):
    del captions_shuffled[ting-i]
    del memes_shuffled[ting-i]
    del token_captions[ting-i]
    del labels_shuffled[ting-i]
    del token_labels[ting-i]

30209


In [25]:
unk_label = []
for i,capt in enumerate(token_labels):
    unk_counter = 0
    for token in capt:
        if token == word2idx['UNK']:
            unk_counter += 1
    if unk_counter >= 2:
        unk_label.append(i)
print(len(unk_label))

733


In [26]:
orig_unk_labels = []
for i in range(len(unk_label)):
    print(labels_shuffled[unk_label[i]])
    orig_unk_labels.append(labels_shuffled[unk_label[i]])
print(len(set(orig_unk_labels)))

pan's labyrinth1
vandinha depressao
vandinha depressao
ranzigen peelaert 
tipichnuy bntu
pan's labyrinth1
tipichnuy bgtu
vandinha depressao
assburgerin superneuvo
tipichnuy bntu
pan's labyrinth1
ranzigen peelaert 
tipichnuy bgtu
vandinha depressao
vandinha depressao
vandinha depressao
pan's labyrinth1
vaverka-perakladczyca
vaverka-perakladczyca
pan's labyrinth1
tipichnuy bgtu
tipichnuy bntu
pan's labyrinth1
tipichnuy bgtu
ranzigen peelaert 
vandinha depressao
ranzigen peelaert 
tipichnuy bntu
pan's labyrinth1
vandinha depressao
pan's labyrinth1
pan's labyrinth1
typacal viewy
pan's labyrinth1
tipichnuy bgtu
pan's labyrinth1
tipichnuy bgeu
tipichnuy bgtu
vandinha depressao
vandinha depressao
vandinha depressao
vandinha depressao
ranzigen peelaert 
vesyolyi-pochvoved
tipichnuy bntu
tipichnuy bntu
tipichnuy bgtu
pan's labyrinth1
pan's labyrinth1
tipichnuy bntu
pan's labyrinth1
pan's labyrinth1
vaverka-perakladczyca
ranzigen peelaert 
pan's labyrinth1
tipichnuy bntu
tipichnuy bgtu
ranzigen 

In [27]:

def _int64_feature(value):
    """Wrapper for inserting an int64 Feature into a SequenceExample proto."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _bytes_feature(value):
    """Wrapper for inserting a bytes Feature into a SequenceExample proto."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _int64_feature_list(values):
    """Wrapper for inserting an int64 FeatureList into a SequenceExample proto."""
    return tf.train.FeatureList(feature=[_int64_feature(v) for v in values])


def _bytes_feature_list(values):
    """Wrapper for inserting a bytes FeatureList into a SequenceExample proto."""
    return tf.train.FeatureList(feature=[_bytes_feature(v) for v in values])

def _floats_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

In [28]:
#ONLY FOR INCEPTION
class ImageDecoder(object):
    """Helper class for decoding images in TensorFlow."""

    def __init__(self):
        # Create a single TensorFlow Session for all image decoding calls.
        self._sess = tf.Session()

        # TensorFlow ops for JPEG decoding.
        self._encoded_jpeg = tf.placeholder(dtype=tf.string)
        self._decode_jpeg = tf.image.decode_jpeg(self._encoded_jpeg, channels=3)

    def decode_jpeg(self, encoded_jpeg):
        image = self._sess.run(self._decode_jpeg,
                               feed_dict={self._encoded_jpeg: encoded_jpeg})
        assert len(image.shape) == 3
        assert image.shape[2] == 3
        return image

In [29]:
#SAVING TRAINING SET (INCEPTION)
import sys
decoder = ImageDecoder()
train_filename = 'train.tfrecords'  # address to save the TFRecords file
# open the TFRecords file
writer = tf.python_io.TFRecordWriter(train_filename)
for i in range(len(memes_shuffled)):
    
    with tf.gfile.FastGFile(memes_shuffled[i], "r") as f:
        encoded_image = f.read()
    
    try:
        decoder.decode_jpeg(encoded_image)
    except (tf.errors.InvalidArgumentError, AssertionError):
        print("Skipping file with invalid JPEG data: %s" % image.filename)
        break

    if not i % 20000:
        print 'Train data: {}/{}'.format(i, len(memes_shuffled))
        sys.stdout.flush()
    
    context = tf.train.Features(feature={
          "train/meme": _bytes_feature(encoded_image), 
      })
    feature_lists = tf.train.FeatureLists(feature_list={
          "train/captions": _int64_feature_list(token_captions[i]),
          #"train/labels": _int64_feature_list(token_labels[i])
      })
    sequence_example = tf.train.SequenceExample(
          context=context, feature_lists=feature_lists)
    
    writer.write(sequence_example.SerializeToString())
    
writer.close()
sys.stdout.flush()

Train data: 0/355487
Train data: 20000/355487
Train data: 40000/355487
Train data: 60000/355487
Train data: 80000/355487
Train data: 100000/355487
Train data: 120000/355487
Train data: 140000/355487
Train data: 160000/355487
Train data: 180000/355487
Train data: 200000/355487
Train data: 220000/355487
Train data: 240000/355487
Train data: 260000/355487
Train data: 280000/355487
Train data: 300000/355487
Train data: 320000/355487
Train data: 340000/355487


In [30]:
#SAVING EVAL SET (INCEPTION)
import sys
decoder = ImageDecoder()
train_filename = 'eval.tfrecords'  # address to save the TFRecords file
# open the TFRecords file
writer = tf.python_io.TFRecordWriter(train_filename)
for i in range(len(eval_memes)):
    
    with tf.gfile.FastGFile(eval_memes[i], "r") as f:
        encoded_image = f.read()
    
    try:
        decoder.decode_jpeg(encoded_image)
    except (tf.errors.InvalidArgumentError, AssertionError):
        print("Skipping file with invalid JPEG data: %s" % image.filename)
        break

    if not i % 20000:
        print 'Train data: {}/{}'.format(i, len(eval_memes))
        sys.stdout.flush()
    
    context = tf.train.Features(feature={
          "train/meme": _bytes_feature(encoded_image), 
      })
    feature_lists = tf.train.FeatureLists(feature_list={
          "train/captions": _int64_feature_list(eval_tokens[i]),
           #"train/labels": _int64_feature_list(token_labels_eval[i])
      })
    sequence_example = tf.train.SequenceExample(
          context=context, feature_lists=feature_lists)
    
    writer.write(sequence_example.SerializeToString())
    
writer.close()
sys.stdout.flush()

Train data: 0/105
